In [1]:
import os
import cv2
import shutil
import random
from ultralytics import YOLO
import torch

In [2]:
def preprocesar_imagen(ruta_origen, ruta_destino, size=(1280, 1280)):
    """
    Lee una imagen, aplica preprocesamiento y la guarda en ruta_destino.
    """
    img = cv2.imread(ruta_origen, cv2.IMREAD_COLOR)
    if img is None:
        print(f"⚠ Error al leer {ruta_origen}")
        return False

    # 1. Aumentar contraste
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    contrastada = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # 2. Reducción de ruido
    denoised = cv2.fastNlMeansDenoisingColored(contrastada, None, 10, 10, 7, 21)

    # 3. Resize
    resized = cv2.resize(denoised, size)

    # 4. Guardar
    cv2.imwrite(ruta_destino, resized)
    return True


In [3]:
def dividir_dataset_con_anotaciones(dataset_original, dataset_dividido,
                                    train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Divide imágenes + anotaciones .txt en train/val/test y preprocesa antes de moverlas.
    """

    # Crear carpetas
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(dataset_dividido, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(dataset_dividido, split, "labels"), exist_ok=True)

    # Listar imágenes originales
    imagenes = [f for f in os.listdir(dataset_original)
                if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    random.shuffle(imagenes)

    n = len(imagenes)
    n_train = int(n * train_ratio)
    n_val = int(n * val_ratio)

    train_imgs = imagenes[:n_train]
    val_imgs = imagenes[n_train:n_train + n_val]
    test_imgs = imagenes[n_train + n_val:]

    # Helper interno
    def mover_con_label(lista_imgs, split_name):
        folder_img = os.path.join(dataset_dividido, split_name, "images")
        folder_lbl = os.path.join(dataset_dividido, split_name, "labels")

        for img in lista_imgs:
            ruta_img = os.path.join(dataset_original, img)
            ruta_lbl = os.path.splitext(ruta_img)[0] + ".txt"

            # Validar que haya anotación
            if not os.path.exists(ruta_lbl):
                print(f"⚠ La imagen {img} NO tiene anotación TXT. Saltando.")
                continue

            # Preprocesar ANTES de mover
            destino_img = os.path.join(folder_img, img)
            preprocesar_imagen(ruta_img, destino_img)

            # Copiar anotación
            destino_lbl = os.path.join(folder_lbl, os.path.splitext(img)[0] + ".txt")
            shutil.copy(ruta_lbl, destino_lbl)

    mover_con_label(train_imgs, "train")
    mover_con_label(val_imgs, "val")
    mover_con_label(test_imgs, "test")

    print("✅ División de dataset completada (con anotaciones).")


In [4]:
def entrenar_yolo(config_yaml, epochs=30, imgsz=1280):
    """
    Entrena YOLOv8n utilizando la configuración YAML del dataset.
    """
    print("Cargando modelo YOLOv8n...")
    model = YOLO("yolov8n.pt")
    print(model)

    print("Iniciando entrenamiento...")
    results = model.train(
        data=config_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=4,
        device="cuda" if torch.cuda.is_available() else "cpu",
        patience=4
    )

    print("🏁 Entrenamiento finalizado.")
    return model, results


In [5]:
def validar_modelo(modelo, ruta_yaml):
    """
    Ejecuta validación sobre el dataset de validación.
    """
    print("📊 Ejecutando validación...")
    metrics = modelo.val(data=ruta_yaml)
    print("📈 Métricas:", metrics)
    return metrics


# Ejecucion de Pipline

In [6]:
#Configuracion de rutas

dataset_original = "../dataset/dataset_imagenes"
dataset_dividido = "./dataset_dividido"
ruta_yaml = "./etiquetas_yolo.yaml"


In [13]:
# Dividir Dataset
dividir_dataset_con_anotaciones(dataset_original, dataset_dividido)

✅ División de dataset completada (con anotaciones).


In [10]:
# Entrenar modelo
modelo, resultados = entrenar_yolo(ruta_yaml, epochs=50)

🚀 Cargando modelo YOLOv8n...
🟢 Iniciando entrenamiento...
New https://pypi.org/project/ultralytics/8.3.234 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.232  Python-3.10.19 torch-2.7.1+cu118 CPU (Intel Core i5-6200U 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./etiquetas_yolo.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_

KeyboardInterrupt: 

In [ ]:
# Validar modelo
validar_modelo(modelo, ruta_yaml)

In [ ]:
import os
import shutil
from ultralytics import YOLO # Importar para verificar el guardado

# --- 1. CONFIGURACIÓN DE RUTAS ---
NOMBRE_RUN = 'train'

# Ruta de origen
RUTA_ORIGEN = os.path.join('runs', 'detect', NOMBRE_RUN, 'weights', 'best.pt')

# Carpeta de destino final
CARPETA_FINAL = 'models'

# Nombre final que tendrá el modelo
NOMBRE_MODELO_FINAL = 'model_yolo8n_v1_best.pt'
RUTA_FINAL = os.path.join(CARPETA_FINAL, NOMBRE_MODELO_FINAL)

# --- 2. PROCESO DE COPIA ---

print(f"Buscando modelo en: {RUTA_ORIGEN}")

# Crear la carpeta de destino si no existe
os.makedirs(CARPETA_FINAL, exist_ok=True)

try:
    # Copiar el archivo 'best.pt' al destino final con un nuevo nombre
    shutil.copy(RUTA_ORIGEN, RUTA_FINAL)

    print(f"\n✅ Modelo copiado exitosamente a: {RUTA_FINAL}")
    print(f"Ruta Absoluta: {os.path.abspath(RUTA_FINAL)}")

    # --- 3. VERIFICACIÓN---
    # Cargar el modelo guardado para asegurar que sea funcional
    # modelo_final = YOLO(RUTA_FINAL)
    # print("\nModelo cargado y verificado: ¡listo para usarse!")

except FileNotFoundError:
    print(f"\n❌ ERROR: No se encontró el modelo en la ruta de origen: {RUTA_ORIGEN}")
    print("Verifica que la carpeta 'runs/detect/' exista y que el NOMBRE_RUN ('train') sea correcto.")